In [1]:
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random


from scipy.ndimage.morphology import binary_erosion as br
from skimage import morphology as skmor

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops

## to calculate
1.	nucleus_MCM_total
2.	outer_MCM_total
3.	inner_MCM_total
4.	nucleus_MCM_hetChrom
5.	outer_MCM_hetChrom
6.	inner_MCM_hetChrom
7.	nucleus_MCM_euChrom
8.	outer_MCM_euChrom
9.	inner_MCM_euChrom


# gathering files
deleting the columns with the signals to recalculate and collect data

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading'

In [5]:
orgfile=f'{bigDir}\\csv_signals_cell_data\signal_data_total_v3.csv'
newfile=f'{bigDir}\\new dataset\cellinfo_signals_v3.csv'

In [6]:
orgdata=pd.read_csv(orgfile)
newdata=pd.read_csv(newfile)

In [7]:
#orgdata=orgdata.drop(columns=['Unnamed: 0'])
orgdata=orgdata.drop(orgdata.iloc[:,39:94],axis=1)


In [8]:
orgdata.head()

,Unnamed: 0,file,original cell name,Amy_mod,movie,x,y,ab1 (heterochromatin),ab1 ch,ab2 (MCM),...,nucleus_Otsu_erosion_outer_9x5_ch2,nucleus_Otsu_erosion_outer_9x5_ch3,nucleus_Otsu_erosion_inner_13x8_volume,nucleus_Otsu_erosion_inner_13x8_ch1,nucleus_Otsu_erosion_inner_13x8_ch2,nucleus_Otsu_erosion_inner_13x8_ch3,nucleus_Otsu_erosion_outer_13x8_volume,nucleus_Otsu_erosion_outer_13x8_ch1,nucleus_Otsu_erosion_outer_13x8_ch2,nucleus_Otsu_erosion_outer_13x8_ch3
0,0,20190625_cell-00.czi,7LE1(B),1130/10,190619-cdc6-dhb-pcna002xy09,172,956,HP1B,1,MCM3,...,847235311.0,6.867781e+09,108435.0,9.745788e+08,425379714.0,3.213620e+09,351463.0,1.816693e+09,882420324.0,7.137711e+09
1,1,20190625_cell-01.czi,7L-E1(T),0,190619-cdc6-dhb-pcna002xy09,158,906,HP1B,1,MCM3,...,833657732.0,7.162939e+09,120724.0,1.040932e+09,468251524.0,3.694105e+09,315813.0,1.598153e+09,802237729.0,6.878110e+09
2,2,20190625_cell-02.czi,7L-E2(B),0,190619-cdc6-dhb-pcna002xy09,832,252,HP1B,1,MCM3,...,935399926.0,7.431494e+09,129190.0,1.430971e+09,633238899.0,4.411502e+09,267850.0,1.751315e+09,844649484.0,6.739183e+09
3,3,20190625_cell-03.czi,7L-E2(T),0,190619-cdc6-dhb-pcna002xy09,888,160,HP1B,1,MCM3,...,892877230.0,7.047608e+09,128365.0,1.246361e+09,588378714.0,3.992517e+09,293263.0,1.638840e+09,816943925.0,6.482227e+09
4,4,20190625_cell-05.czi,7L-G2,0,190619-cdc6-dhb-pcna002xy09,172,324,HP1B,1,MCM3,...,961789578.0,1.280485e+10,296285.0,3.478092e+09,900348539.0,9.504919e+09,517987.0,3.527791e+09,937550088.0,1.211177e+10


In [9]:
newdata=newdata.drop(columns=['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1'])
newdata=newdata.drop(newdata.iloc[:,19:60], axis=1)

In [10]:
newdata.head()

,file,original cell name,movie,last frame movie position x,last frame movie position y,ab1 (MCM),ab1 ch,ab2 (heterochromatin),ab2 ch,DAPI ch,DIC ch,category,molecular age,physical age [min],imaging core,cell id for tracking,comment,molecular age 10,path
0,20191203_cell-00.czi,20191125-01-17,191125pcnat_dhbr_cdc6vxy01,678.301,200.931,MCM3,1,HP1beta,2,3,4,middleG1,0.444,340.0,HIC,17,NaN,0.311,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
1,20191203_cell-01.czi,20191125-01-20,191125pcnat_dhbr_cdc6vxy01,1552.152,240.479,MCM3,1,HP1beta,2,3,4,G2,1.373,NaN,HIC,20,110 min after s,2.036,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
2,20191203_cell-02.czi,20191125-01-23,191125pcnat_dhbr_cdc6vxy01,1131.436,318.530,MCM3,1,HP1beta,2,3,4,G2,1.371,910.0,HIC,23,30 min after S,1.960,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
3,20191203_cell-03.czi,20191125-01-25,191125pcnat_dhbr_cdc6vxy01,1650.585,324.968,MCM3,1,HP1beta,2,3,4,earlyG1,0.704,30.0,HIC,25,NaN,0.632,Z:\CookLab\Liu\20190816_organizedData_MCM_load...
4,20191203_cell-04.czi,20191125-01-32,191125pcnat_dhbr_cdc6vxy01,1466.687,426.815,MCM3,1,HP1beta,2,3,4,middleG1,0.345,250.0,HIC,32,NaN,0.225,Z:\CookLab\Liu\20190816_organizedData_MCM_load...


# 2020 dataset calculations

In [11]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200207_new data'

In [12]:
file=f'{bigDir}\\cellinfo_200203_v7.csv'
data=pd.read_csv(file)

# Geminin dataset calculations

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200217_geminin'

In [3]:
file=f'{bigDir}\\200217_cellinfo_v3.csv'
data=pd.read_csv(file)

# H3K9ME3 

In [4]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200604_h3k9me3'

In [5]:
file=f'{bigDir}\\cellinfo_200604v3.csv'
data=pd.read_csv(file)

# H3K9ME3 2nd dataset

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200615_h3k9me3'

In [4]:
file=f'{bigDir}\\cellinfo_200615v2.csv'
data=pd.read_csv(file)

# mcm calculations

In [6]:
def calculate_mcm_signals (name, data):
    
    ## if the given dataset is new, the mcm channel is 0
    ## else, mcm channel is 1 (for original dataset)
    if name=="new":
        mcm=0
    else:
        mcm=1
    i=0
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)
    for i,myCell in islice(data.iterrows(), i,None):
        # i = index of the row
        # myCell the entire row of the that index i = data.iloc[i]
        
        ## opening cell image
        try:
            myImage=imread(myCell.path) 
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)
        
        ## setting arrays with names 
        
        ### for the column names
        pixels=['total','het','eu']
        components=['nuc','inner','outer']
        
        ### for image names
        dirs=['segmentation_nucleus_Otsu','erosion_nuc_mask','erosion_nuc_mask',
              'segmentation_20','segmentation_20_erosion','segmentation_20_erosion',
             'segmentation_20_eu','segmentation_20_eu_erosion','segmentation_20_eu_erosion',]

        files=['_nucleus.tif','_eroded_9^5_inner_mask_20.tif','_eroded_9^5_outer_mask_20.tif',
                   '_hetChrom.tif', '_hetChrom_inner.tif', '_hetChrom_outer.tif',
                   '_euChrom.tif', '_euChrom_inner.tif','_euChrom_outer.tif']

        ## making a dummy dataframe to store the signal calculations
        df=pd.DataFrame(columns=['nuc_vol_total','inner_vol_total','outer_vol_total',
                                 'nuc_mcm_total','inner_mcm_total','outer_mcm_total',
                                'nuc_vol_het','inner_vol_het','outer_vol_het',
                                 'nuc_mcm_het','inner_mcm_het','outer_mcm_het',
                                 'nuc_vol_eu','inner_vol_eu','outer_vol_eu',
                                'nuc_mcm_eu','inner_mcm_eu', 'outer_mcm_eu'])

        ## signals calculated
        for k in range(len(pixels)): # 0 to 2 - 3 values for total, het, eu
            for j in range(len(components)): 
                # 0 to 2 
                           
                # 1 loop
                # total (k=0) , nuc  (j=0)
                # total (k=0), inner (j=1)
                # total (k=0), outer (j=2)
                           
                ## copying image every time new signal is calculated
                myIm=myImage.copy()
                myChannel=myIm[:,mcm,:,:] 
                
                ## column name set up based on the component and the pixels
                currColName=f'{components[j]}_mcm_{pixels[k]}' # nuc_mcm_total
                
                ## opening mask images
                ### the indexing is in such way so that the image opened matches up with the name being used for calculation
                direc=myCell.path.replace('data_tiff',dirs[(j)+(k*3)]) 
                    # dir[0+0] = dir[0]= segmentation_nuclues_otsu
                    # dirs[1+0] = dir[1] = erosion_nuc_mask
                    # dir[2+0] = dir[2] = erosion_nuc_mask
                
                           
                try:
                    direc=direc.replace('.tif',files[(k*3)+(j)])
                    mask=imread(direc)  
                except:
                    direc=direc.replace('_nucleus.tiff',files[(k*3)+(j)])
                    mask=imread(direc)

                ## changing mask type to binary
                mask=mask.astype(bool)
                
                ## masking the image
                myChannel[~mask]=0
                
                ## calculating the signal
                df[f'{currColName}']=myChannel.sum(-1).sum(-1)
                           
                ## calculating the volume of the pixels
                currColName=f'{components[j]}_vol_{pixels[k]}'
                df.loc[0,f'{currColName}']=np.sum(mask)
        
        ## saving the calculated signals to original/big dataframe
        for myMeasurement in df.columns:
            myString=f"data.loc[i,'{myMeasurement}']=np.sum(df.{myMeasurement})"
            exec(myString)
        

        progBar.value=progBar.value+1
        

In [10]:
## running the function for new dataset
calculate_mcm_signals ("new", newdata) # mcm channel  = 0, thats why 'new'

IntProgress(value=0, description='Progress:', max=324)

In [13]:
## testing to see if the calculated signals have results as expected
print(
(np.sum(newdata['nuc_mcm_het']+newdata['nuc_mcm_eu']))/np.sum(newdata['nuc_mcm_total']),
(np.mean(newdata['inner_mcm_het'])+np.mean(newdata['inner_mcm_eu']))/np.mean(newdata['inner_mcm_total']),
(np.sum(newdata['outer_mcm_het']+newdata['outer_mcm_eu']))/np.sum(newdata['outer_mcm_total']))

1.00000000069842 0.9999999981338163 1.0000000045880304


In [12]:
print(
(np.sum(newdata['inner_mcm_het']+newdata['outer_mcm_het']))/np.sum(newdata['nuc_mcm_het']),
(np.mean(newdata['outer_mcm_eu'])+np.mean(newdata['inner_mcm_eu']))/np.mean(newdata['nuc_mcm_eu']),
(np.sum(newdata['nuc_mcm_het'])+np.sum(newdata['nuc_mcm_eu']))/np.sum(newdata['nuc_mcm_total']),
(np.sum(newdata['inner_mcm_total']+newdata['outer_mcm_total'])/np.sum(newdata['nuc_mcm_total'])))

1.0000000008276106 1.0000000000969513 1.00000000069842 0.9999999976847376


In [14]:
## saving the dataset with calculated signals
newdata.to_csv(f'{bigDir}\\new dataset\signals_new_v1.csv')

### mcm calculations for 2020 dataset

In [16]:
## running the function for new dataset
calculate_mcm_signals ("2020", data) # MCM channel = 2nd 

IntProgress(value=0, description='Progress:', max=146)

In [17]:
## testing to see if the calculated signals have results as expected
print(
(np.sum(data['nuc_mcm_het']+data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.mean(data['inner_mcm_het'])+np.mean(data['inner_mcm_eu']))/np.mean(data['inner_mcm_total']),
(np.sum(data['outer_mcm_het']+data['outer_mcm_eu']))/np.sum(data['outer_mcm_total']))

1.0 1.0000000000000002 1.0


In [18]:
print(
(np.sum(data['inner_mcm_het']+data['outer_mcm_het']))/np.sum(data['nuc_mcm_het']),
(np.mean(data['outer_mcm_eu'])+np.mean(data['inner_mcm_eu']))/np.mean(data['nuc_mcm_eu']),
(np.sum(data['nuc_mcm_het'])+np.sum(data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.sum(data['inner_mcm_total']+data['outer_mcm_total'])/np.sum(data['nuc_mcm_total'])))

1.0 1.0000000000000002 1.0 1.0


In [20]:
## saving the dataset with calculated signals
data.to_csv(f'{bigDir}\\cellinfo_200203_v7.csv')

### mcm calculations for geminin dataset

In [5]:
## running the function for geminin dataset
calculate_mcm_signals ("new", data) # MCM channel = 2nd 

IntProgress(value=0, description='Progress:', max=145)

In [6]:
## testing to see if the calculated signals have results as expected
print(
(np.sum(data['nuc_mcm_het']+data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.mean(data['inner_mcm_het'])+np.mean(data['inner_mcm_eu']))/np.mean(data['inner_mcm_total']),
(np.sum(data['outer_mcm_het']+data['outer_mcm_eu']))/np.sum(data['outer_mcm_total']))

1.0 1.0 1.0


In [7]:
print(
(np.sum(data['inner_mcm_het']+data['outer_mcm_het']))/np.sum(data['nuc_mcm_het']),
(np.mean(data['outer_mcm_eu'])+np.mean(data['inner_mcm_eu']))/np.mean(data['nuc_mcm_eu']),
(np.sum(data['nuc_mcm_het'])+np.sum(data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.sum(data['inner_mcm_total']+data['outer_mcm_total'])/np.sum(data['nuc_mcm_total'])))

1.0 1.0 1.0 1.0


In [8]:
## saving the dataset with calculated signals
data.to_csv(file.replace('v3','v4'))

### mcm calculations for h3k9me3 dataset

In [8]:
## running the function for geminin dataset
calculate_mcm_signals ("h3k9me3", data) # MCM channel = 2nd 

IntProgress(value=0, description='Progress:', max=78)

In [9]:
## testing to see if the calculated signals have results as expected
print(
(np.sum(data['nuc_mcm_het']+data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.mean(data['inner_mcm_het'])+np.mean(data['inner_mcm_eu']))/np.mean(data['inner_mcm_total']),
(np.sum(data['outer_mcm_het']+data['outer_mcm_eu']))/np.sum(data['outer_mcm_total']))

1.0 1.0 1.0


In [10]:
print(
(np.sum(data['inner_mcm_het']+data['outer_mcm_het']))/np.sum(data['nuc_mcm_het']),
(np.mean(data['outer_mcm_eu'])+np.mean(data['inner_mcm_eu']))/np.mean(data['nuc_mcm_eu']),
(np.sum(data['nuc_mcm_het'])+np.sum(data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.sum(data['inner_mcm_total']+data['outer_mcm_total'])/np.sum(data['nuc_mcm_total'])))

1.0 1.0 1.0 1.0


In [11]:
## saving the dataset with calculated signals
data.to_csv(file.replace('v3','v4'))

### mcm calculations for h3k9me3 2nd dataset

In [7]:
## running the function for geminin dataset
calculate_mcm_signals ("h3k9me3", data) # MCM channel = 2nd 

IntProgress(value=0, description='Progress:', max=88)

In [8]:
## testing to see if the calculated signals have results as expected
print(
(np.sum(data['nuc_mcm_het']+data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.mean(data['inner_mcm_het'])+np.mean(data['inner_mcm_eu']))/np.mean(data['inner_mcm_total']),
(np.sum(data['outer_mcm_het']+data['outer_mcm_eu']))/np.sum(data['outer_mcm_total']))

1.0 0.9999999999999998 1.0


In [9]:
print(
(np.sum(data['inner_mcm_het']+data['outer_mcm_het']))/np.sum(data['nuc_mcm_het']),
(np.mean(data['outer_mcm_eu'])+np.mean(data['inner_mcm_eu']))/np.mean(data['nuc_mcm_eu']),
(np.sum(data['nuc_mcm_het'])+np.sum(data['nuc_mcm_eu']))/np.sum(data['nuc_mcm_total']),
(np.sum(data['inner_mcm_total']+data['outer_mcm_total'])/np.sum(data['nuc_mcm_total'])))

1.0 1.0 1.0 1.0


In [12]:
## saving the dataset with calculated signals
data.to_csv(file.replace('v2','v3'))